# Mini project 6 extra

В рамках дополнительного проекта попробуем освоить API, позволяющий получать данные о музейных коллекциях. Это задание может пригодиться для большего понимания того, какими способами можно собирать и обрабатывать данные, например, для собственного проекта или другой рабочей задачи. Также вы можете более глубоко изучить документацию, чтобы потом исследовать другие интересные для себя вопросы по полученным данным

Описание API 

Документация: https://metmuseum.github.io

Ключ: для использования выбранного API (на момент написания задания) ключ не требуется

Всего есть 4 основных раздела:

    Объекты (Objects)
    Объект (Object) – данные об объекте (например, о картине)
    Департаменты (Departments) – данные о департаменте/разделе (например, "Современное искусство")
    Поиск (Search) – используется для поиска

В рамках задач поработаем с блоками Object и Search.

С более подробным описанием каждого доступного параметра можно ознакомиться в документации.

#### 1. Для начала, импортируйте нужные библиотеки

In [3]:
import requests
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

#### 2. Сам запрос к API должен состоять из нескольких параметров. Пример обращения к поиску (Search):

#### Предположим, ваша задача – найти все произведения, связанные с подсолнухами. Выполните поиск по слову "sunflowers". Внимательно посмотрите на формат ответа и сохраните id полученных объектов в список sunflower_ids.

In [4]:
r = requests.get('https://collectionapi.metmuseum.org/public/collection/v1/search?q=sunflowers')
sunflowers = r.json()

In [5]:
sunflower_ids = sunflowers['objectIDs']
search_ids = [436524, 2032, 20149, 16822570, 437329]

In [6]:
sunflowers_search = [i for i in search_ids if i in sunflower_ids]
sunflowers_search

[436524, 2032, 437329]

#### 2. Отлично! На предыдущем шаге мы получили результаты поиска. Следущий этап – собрать информацию об этих объектах.

Пример текста запроса для получения информации об одном объекте (Object):

r_test = requests.get(f'https://collectionapi.metmuseum.org/public/collection/v1/objects/OBJECT_ID') 
obj_test = r_test.json()

In [9]:
# запрос из задачи, подставили id объекта
r_test = requests.get(f'https://collectionapi.metmuseum.org/public/collection/v1/objects/437980')  
obj_test = r_test.json()

GalleryNumber 825

artistDisplayName Vincent van Gogh

objectName Painting

title Cypresses

department European Paintings

culture ''

objectID 437980

#### 3. Теперь напишите цикл, с помощью которого будет собрана информация об объектах, которые были получены на 2 шаге. Результаты запишите в датафрейм, а именно – следующие параметры:

        objectID – id объекта
        title – название
        artistDisplayName – автор
        department – департамент
        objectBeginDate – дата (начало)
        objectEndDate – дата (конец)
        period – название периода
        objectName – название/категория объекта
        culture – культура
   
#### Сколько объектов относятся к японской культуре? (culture –Japan)

In [10]:
object_df = pd.DataFrame()

In [12]:
for object_id in sunflower_ids:
    r = requests.get(f'https://collectionapi.metmuseum.org/public/collection/v1/objects/{object_id}')
    obj_tmp = r.json()

In [13]:
try:
    obj_tmp_df = pd.DataFrame({'objectID'         : obj_tmp['objectID'],
                              'title'             : [obj_tmp['title']],
                              'artistDisplayName' : [obj_tmp['artistDisplayName']],
                              'department'        : [obj_tmp['department']], 
                              'objectBeginDate'   : [obj_tmp['objectBeginDate']], 
                              'objectEndDate'     : [obj_tmp['objectEndDate']], 
                              'period'            : [obj_tmp['period']], 
                              'objectName'        : [obj_tmp['objectName']], 
                              'culture'           : [obj_tmp['culture']]
                             })
    # соединение
    object_df = pd.concat([object_df, obj_tmp_df])
except KeyError:
    pass

In [14]:
object_df = pd.concat([object_df, obj_tmp_df])

In [15]:
object_df.query("culture == 'Japan'").shape[0]

0

#### 4. Теперь попробуем построить более сложный запрос с помощью секции "Search"! На этот раз параметры необходимо передать через "&", например, departmentId (id департамента – 1) и q (ключевое слово – cat):

#### Выполните поисковый запрос для департамента "Asian art" (departmentId = 6) с тегом "cat" и запишите id объектов в cat_ids. Используйте цикл из шага 4, чтобы получить данные о первой 1000 объектов из списка.

Note: выполнение цикла может занять некоторое время (~5 минут), поэтому проверять его работоспособность лучше на более маленькой выборке!

##### В качестве ответа укажите, в каком периоде (period) было выполнено наибольшее количество работ, представленных в музее?

In [16]:
r    = requests.get('https://collectionapi.metmuseum.org/public/collection/v1/search?departmentId=6&q=cat')
cats = r.json()

In [17]:
cat_ids = cats['objectIDs'] 


In [18]:
for object_id in cat_ids[:1000]:  # берем только 1000 объектов, как указано в задании
    # информация об объекте
    r = requests.get(f'https://collectionapi.metmuseum.org/public/collection/v1/objects/{object_id}')
    obj_tmp = r.json()

    # формируем строчку для объекта
    try:
        obj_tmp_df = pd.DataFrame({'objectID' : obj_tmp['objectID'],
                                 'title' : [obj_tmp['title']],
                                 'artistDisplayName': [obj_tmp['artistDisplayName']],
                                 'department' : [obj_tmp['department']], 
                                 'objectBeginDate' : [obj_tmp['objectBeginDate']], 
                                 'objectEndDate' : [obj_tmp['objectEndDate']], 
                                 'objectName' : [obj_tmp['objectName']], 
                                 'culture': [obj_tmp['culture']],
                                 'period': [obj_tmp['period']]
                            })
        # приклеиваем
        object_df = pd.concat([object_df, obj_tmp_df])
    except KeyError:
        continue

In [19]:
object_df.period.value_counts().head(5)

period
Edo period (1615–1868)          272
                                146
Qing dynasty (1644–1911)         63
Ming dynasty (1368–1644)         47
Muromachi period (1392–1573)     30
Name: count, dtype: int64

Timurid period (1370–1507)

##### Edo period (1615–1868)

Tang dynasty (618–907)

Meiji period (1868–1912)

Период чаще всего не указан